In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
 
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def augment(image,label):
    return image,(tf.reshape(image,[-1]),label)

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(
    augment, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

x= tf.keras.layers.Input((28,28))
z= tf.keras.layers.Flatten(input_shape=(28, 28),name="zz")(x)
y=tf.keras.layers.Dense(128, activation='relu')(z)
y=  tf.keras.layers.Dense(10,name="prediction")(y)
model= tf.keras.Model(x,[z,y])
def l2_loss(model):
    return lambda :tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])

class L2LossHistoryCallback(tf.keras.callbacks.Callback):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.l2_loss=l2_loss(model)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs["l2_loss"] = float(self.l2_loss())


model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss={"prediction":tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),"zz":tf.keras.losses.MeanSquaredError()},
    metrics={"prediction":tf.keras.metrics.SparseCategoricalAccuracy()}
)
model.add_loss({"l2":l2_loss(model)})

t=model.fit(
    ds_train,
    epochs=3,
    validation_data=ds_test,callbacks=[L2LossHistoryCallback(model)]
)


Epoch 1/3


2023-06-09 02:43:17.101709: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-06-09 02:43:17.102158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


462/469 [============================>.] - ETA: 0s - loss: 8.9751 - zz_loss: 0.0000e+00 - prediction_loss: 2.2122 - prediction_sparse_categorical_accuracy: 0.4281

2023-06-09 02:43:25.601812: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-06-09 02:43:25.602947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]


469/469 [==============================] - 9s 6ms/step - loss: 8.8791 - zz_loss: 0.0000e+00 - prediction_loss: 2.2135 - prediction_sparse_categorical_accuracy: 0.4234 - val_loss: 29.3690 - val_zz_loss: 27.0660 - val_prediction_loss: 2.3023 - val_prediction_sparse_categorical_accuracy: 0.1028 - l2_loss: 7.7668e-04
Epoch 2/3
469/469 [==============================] - 2s 4ms/step - loss: 2.3026 - zz_loss: 0.0000e+00 - prediction_loss: 2.3023 - prediction_sparse_categorical_accuracy: 0.1108 - val_loss: 29.3685 - val_zz_loss: 27.0660 - val_prediction_loss: 2.3023 - val_prediction_sparse_categorical_accuracy: 0.1135 - l2_loss: 1.4205e-04
Epoch 3/3
469/469 [==============================] - 2s 4ms/step - loss: 2.3025 - zz_loss: 0.0000e+00 - prediction_loss: 2.3023 - prediction_sparse_categorical_accuracy: 0.1107 - val_loss: 29.3685 - val_zz_loss: 27.0660 - val_prediction_loss: 2.3024 - val_prediction_sparse_categorical_accuracy: 0.1135 - l2_loss: 9.3754e-05


In [6]:
model.save("model.pb")

INFO:tensorflow:Assets written to: model.pb/assets


INFO:tensorflow:Assets written to: model.pb/assets


In [13]:
loaded_module=tf.saved_model.load("model.pb")

In [22]:
x,y=next(iter(ds_test))

In [12]:
t.history

{'loss': [8.879067420959473, 2.30257511138916, 2.302513837814331],
 'zz_loss': [0.0, 0.0, 0.0],
 'prediction_loss': [2.21354079246521, 2.3023481369018555, 2.3023171424865723],
 'prediction_sparse_categorical_accuracy': [0.4234333336353302,
  0.11081666499376297,
  0.11071666330099106],
 'val_loss': [29.369009017944336, 29.368450164794922, 29.368450164794922],
 'val_zz_loss': [27.06597900390625, 27.06597900390625, 27.06597900390625],
 'val_prediction_loss': [2.302251100540161,
  2.3023297786712646,
  2.3023808002471924],
 'val_prediction_sparse_categorical_accuracy': [0.10279999673366547,
  0.11349999904632568,
  0.11349999904632568],
 'l2_loss': [<tf.Tensor: shape=(), dtype=float32, numpy=0.0007766833>,
  <tf.Tensor: shape=(), dtype=float32, numpy=0.0001420502>,
  <tf.Tensor: shape=(), dtype=float32, numpy=9.375398e-05>]}

<bound method Layer.add_loss of <keras.engine.sequential.Sequential object at 0x7ff7e1ad6e90>>

In [54]:
tf.res(tf.constant([1,2]))

AttributeError: module 'tensorflow' has no attribute 'flatten'

In [27]:
model.predict(tf.ones([1,28,28,1]))

1/1 [==============================] - 0s 32ms/step


array([[-26.798069  , -16.96833   ,   2.0264852 ,  29.349308  ,
        -50.946102  ,  26.800014  ,   4.567534  ,   5.227916  ,
         -0.28305477,  -8.013782  ]], dtype=float32)

In [44]:
tf.keras.backend.clear_session()

In [16]:
model.arch="mlp"

In [17]:
model.save("test.pb")

INFO:tensorflow:Assets written to: test.pb/assets


INFO:tensorflow:Assets written to: test.pb/assets


In [18]:
loaded_model=tf.keras.models.load_model("test.pb")

In [20]:
loaded_model

In [141]:
class DumpyDataset:

    """Iterator that counts upward forever."""

    def __init__(self, start=0):
        self.num = start

    def __iter__(self):
        return self

    def __next__(self):
        num = self.num
        self.num += 1
        return tf.zeros([5,5],dtype=tf.float32),tf.ones(1)



In [142]:
S=DumpyDataset()
specs=(tf.TensorSpec(shape=(5,5)),tf.TensorSpec(shape=()))

dataset=tf.data.Dataset.from_generator(lambda :S,output_signature=specs).batch(32)

In [143]:
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(25,activation="sigmoid"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
model.compile(optimizer="adam",loss="mean_squared_error")

In [161]:
model.fit(x=S,epochs=3,steps_per_epoch=200)

Epoch 1/3
 42/200 [=====>........................] - ETA: 0s - loss: 9.5807e-04

2023-06-09 09:07:39.855540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


200/200 [==============================] - 1s 4ms/step - loss: 7.9380e-04
Epoch 2/3
200/200 [==============================] - 1s 4ms/step - loss: 5.1179e-04
Epoch 3/3
200/200 [==============================] - 1s 4ms/step - loss: 3.5767e-04


In [82]:
next(iter(dataset))

(<tf.Tensor: shape=(1, 5, 5), dtype=float32, numpy=
 array([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>)

In [104]:
next(next(S))

(<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], dtype=float32)>,
 1)

In [145]:
model

In [153]:
C=tf.train.Checkpoint(model)

In [154]:
C.save("GJ")

'GJ-1'

In [156]:
C.save("GJ")

'GJ-3'

In [152]:
C.restore("GJ-2")

In [157]:
D=tf.train.Checkpoint(model)

In [163]:
model.trainable_variables

[<tf.Variable 'dense_22/kernel:0' shape=(5, 25) dtype=float32, numpy=
 array([[-3.10127914e-01, -1.04571402e-01, -4.00420576e-01,
         -4.16880757e-01,  1.77367687e-01,  4.04328465e-01,
         -2.44916141e-01, -3.94790292e-01, -1.09520465e-01,
          1.61812842e-01,  3.88856113e-01, -7.30258524e-02,
          4.27775919e-01, -1.33364052e-01,  1.86398208e-01,
         -3.68464291e-01, -1.14259690e-01, -1.55187428e-01,
          4.17616785e-01, -3.98608506e-01,  3.81016612e-01,
         -4.38223064e-01, -4.25603449e-01,  3.25418115e-01,
         -3.29489350e-01],
        [ 4.04741406e-01,  2.62748361e-01, -2.96535730e-01,
         -4.86840010e-02,  3.83847237e-01, -2.45123625e-01,
         -1.54168606e-02, -3.03385258e-01,  1.06430411e-01,
          9.70088840e-02,  1.10525489e-02, -2.39927948e-01,
          3.62334549e-01,  3.01063478e-01,  2.88561106e-01,
          3.49425197e-01,  6.32990599e-02,  2.19262183e-01,
         -1.53903157e-01, -3.98854911e-01,  3.33956778e-01,
   